In [1]:
import cv2
import json
import math
import os
import os.path
import re
import tempfile
from matplotlib import pyplot
from util import quickstart as util
from trainer import modelDesign as training_model

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/home/jjwong0915/Documents/drone/depth_prediction/.venv/lib/python3.7/imp.py", line 342, in l

## Parameters Setting

In [ ]:
# 欲載入 pretrain weight 的路徑
MODEL_WEIGHT_PATH = '/home/jjwong0915/Documents/drone/depth_prediction/model/airsim_2020-01-14-16-56_epoch-006_loss-9.0455.h5'

# 資料路徑
DATA_PATH = '/home/jjwong0915/Documents/drone/depth_prediction/data/random/2019-12-12-04-54/image'
RESULT_PATH = '/home/jjwong0915/Documents/drone/depth_prediction/result/2019-12-05-05-49'

# Model Input 解析度設定
INPUT_HEIGHT = 320
INPUT_WIDTH = 384

# Video Output 解析度設定
OUTPUT_HEIGHT = 1440
OUTPUT_WIDTH = 1920

## Build Model

In [ ]:
# 建立 model layer 架構
model = training_model.MNv2_segment_depth_multiloss_model(
    inputShape = (INPUT_HEIGHT, INPUT_WIDTH, 3), 
    alpha = 0.35, 
    expansion_factor = 6, 
    depth_multiplier = 1, 
    lock_backend_weights = False,
    CLASSES = 6,
)

# 載入 model weight
model.load_weights(MODEL_WEIGHT_PATH)

model.summary()

## Predict the Image and Save Result

In [ ]:
# iterate each dataset
for folder_name in os.listdir(DATA_PATH):
    # prepare result image folder
    result_image_folder_path = os.path.join(RESULT_PATH, 'image', folder_name)
    if not os.path.exists(result_image_folder_path):
        os.makedirs(result_image_folder_path)
    # iterate each image
    folder_path = os.path.join(DATA_PATH, folder_name) 
    for file_name in os.listdir(folder_path):
        # 讀取影像
        scene = util.read_image(os.path.join(folder_path, file_name), shape=(INPUT_HEIGHT,INPUT_WIDTH), rand=0)
        segment = util.read_image(
            os.path.join(folder_path.replace('image', 'seg'), file_name), 
            shape = (INPUT_HEIGHT,INPUT_WIDTH), 
        )
        # 使用 image 來預測出 depth map 和 semantic segmentation
        predict_depth, predict_segment, predict_risk_index, predict_depth_pfm = util.getResult(model, scene, (INPUT_HEIGHT, INPUT_WIDTH))
        print(predict_risk_index)
        # 儲存結果
        image_path = os.path.join(RESULT_PATH, 'image', folder_name, file_name)
        util.saveResult([scene, segment, predict_depth[0]], predict_risk_index, image_path, (OUTPUT_WIDTH/300, OUTPUT_HEIGHT/300))
    print('predicted dataset {}'.format(folder_name))
            

# Convert Images into Video 

In [ ]:
# prepare result video folder
result_video_folder_path = os.path.join(RESULT_PATH, 'video')
if not os.path.exists(result_video_folder_path):
    os.makedirs(result_video_folder_path)
# iterate result images
result_image_folder =  os.path.join(RESULT_PATH, 'image')
for folder_name in os.listdir(result_image_folder):
    # create result video
    video_path = os.path.join(result_video_folder_path, folder_name+'.mp4')
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'MP4V'), 10, (OUTPUT_WIDTH, OUTPUT_HEIGHT))
    # iterate each image
    for file_name in sorted(os.listdir(os.path.join(result_image_folder, folder_name)), key=lambda x: int(re.split('\\-|\\.', x)[-2])):
        # 儲存結果
        image_path = os.path.join(result_image_folder, folder_name, file_name)
        image = cv2.resize(cv2.imread(image_path), (OUTPUT_WIDTH, OUTPUT_HEIGHT))
        video.write(image)
    video.release()
    print('{} created'.format(video_path))

In [ ]:
weights = model.layers[-1].get_weights()
print(weights)